In [1]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')
import torch
torch.set_printoptions(linewidth=200)
import seaborn as sns
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
import yaml
from ionpy.util import Config
from ese.scripts.utils import get_option_product
# Results loader object does everything
from ionpy.analysis import ResultsLoader
from pathlib import Path
root = Path("/storage/vbutoi/scratch/ESE")
# Define some useful paths.
scratch_root = Path("/storage/vbutoi/scratch/ESE")
code_root = Path("/storage/vbutoi/projects/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 
# For using yaml configs.
%load_ext yamlmagic

In [2]:
%%yaml calibration_config

data:
  preload: '?' 
  iters_per_epoch: '?' 

dataloader:
  batch_size: 4 
  num_workers: 2
  pin_memory: True 

optim: 
  _class: torch.optim.Adam
  lr: 1.0e-4
  weight_decay: 0.0 

train:
  epochs: 500 
  eval_freq: 10 
  pretrained_dir: '?'
  pretrained_select_metric: 'val-dice_score'

loss_func: 
  _class: ionpy.loss.PixelCELoss 
  from_logits: True 
  batch_reduction: 'mean' 
  ignore_index: 0 # Foreground calibrators

<IPython.core.display.Javascript object>

In [3]:
%%yaml callbacks_cfg

callbacks:
  step:
    - ese.experiment.callbacks.ShowPredictions
  epoch:
    - ese.experiment.callbacks.WandbLogger
    - ionpy.callbacks.ETA
    - ionpy.callbacks.JobProgress
    - ionpy.callbacks.TerminateOnNaN
    - ionpy.callbacks.PrintLogged
    - ionpy.callbacks.ModelCheckpoint:
        monitor: ece_loss 
        phase: val

<IPython.core.display.Javascript object>

In [4]:
%%yaml calibrator_model_cfg  

model:
  _class: '?'
  neighborhood_width: 3

<IPython.core.display.Javascript object>

In [5]:
exp_name = '02_18_24_WMH_MinLabelFG_Calibrators' 
group_dict = {
    "dataset": "WMH",
    "exp_group": exp_name,
    "base_models_group": "01_08_24_WMH_Ensemble",
    "preload": False
}

# exp_name = '01_26_24_CityScapes_Ensemble' 
# group_dict = {
#     "dataset": "CityScapes",
#     "exp_group": exp_name,
#     "base_models_group": "01_25_24_CityScapes_Dice",
#     "preload": False
# }

In [6]:
from ese.experiment.analysis.analysis_utils.submit_utils import get_ese_calibration_configs

option_set = get_ese_calibration_configs(
    group_dict=group_dict,
    calibrators=[
        # "Vanilla",
        # "TempScaling",
        # "VectorScaling",
        # "DirichletScaling",
        "LTS",
        "NectarScaling",
        "NS_V2"
    ],
    base_options=None
)

In [7]:
# Load the inference cfg from local.
##################################################
cal_cfg_root = code_root / "ese" / "experiment" / "configs" / "calibrate"
##################################################
with open(cal_cfg_root / f"{group_dict['dataset']}.yaml", 'r') as file:
    dataset_cal_cfg = yaml.safe_load(file)

# Assemble base config
base_cfg = Config(calibration_config).update([calibrator_model_cfg, callbacks_cfg, dataset_cal_cfg])

# Get the configs
cfgs = get_option_product(exp_name, option_set, base_cfg)

In [8]:
len(cfgs)

12

## Running Jobs

In [9]:
from ese.experiment.experiment import run_ese_exp, submit_ese_exps, PostHocExperiment

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_outputs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [10]:
# ####### Run individual jobs
# run_ese_exp(
#     config=cfgs[0], 
#     experiment_class=PostHocExperiment,
#     run_name='debug',
#     show_examples=True,
#     track_wandb=False,
#     gpu='3',
# )

In [11]:
###### Run Batch Jobs
submit_ese_exps(
    config_list=cfgs,
    experiment_class=PostHocExperiment,
    track_wandb=True,
    available_gpus=['0', '1', '2', '3']
    # available_gpus=['4', '5', '6', '7']
)

Submitted job id: 1189386 on gpu: 0.
Submitted job id: 1189457 on gpu: 1.
Submitted job id: 1189564 on gpu: 2.
Submitted job id: 1189677 on gpu: 3.
Submitted job id: 1189853 on gpu: 0.
Submitted job id: 1190010 on gpu: 1.
Submitted job id: 1190159 on gpu: 2.
Submitted job id: 1190374 on gpu: 3.
Submitted job id: 1190652 on gpu: 0.
Submitted job id: 1190814 on gpu: 1.
Submitted job id: 1191301 on gpu: 2.
Submitted job id: 1191462 on gpu: 3.
